In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

D:\Anaconda3\envs\crawler\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\envs\crawler\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\envs\crawler\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\envs\crawler\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: P

In [2]:
#this is data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
#hyperparameters
lr = 0.001
training_iters = 100000  #循环多少次
batch_size = 128
#display_step = 10

n_inputs = 28 #MNIST data input (img shape:28*28)
n_steps = 28 #time steps 
n_hidden_units = 128 #neurons in hidden layer
n_classes = 10 #Mnist classes(0-9 digits)

#tf Graph input
x = tf.placeholder(tf.float32, [None,n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None,n_classes])


#define weights
weights = {
    #（28*128）
    'in':tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),
    #(128,10)
    'out':tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}
biases = {
    #(128,)
    'in':tf.Variable(tf.constant(0.1, shape=[n_hidden_units,])),
    #(10.)
    'out':tf.Variable(tf.constant(0.1, shape=[n_classes,]))
}

In [4]:
def RNN(X, weights, bias):
    # hidden layers for input cell
    ###############################################
    #X(128 batchm28 steps(行),28 inputs(piexl)) 
    #===> (128*28,28 inputs)
    X = tf.reshape(X, [-1,n_inputs])
    #X_in ===> (128batch*28 steps, 128 hidden)
    X_in = tf.matmul(X, weights['in']+biases['in'])
    #X_in ===> (128batch，28 steps, 128 hidden)
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_units])
    
    #cell
    ################################################
    #state_is_tuple 看是否生成这样的一个元组
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units,forget_bias=1.0, state_is_tuple=True)  
    #lstm cell is divided into two parts(c_state, m_state)
    _init_state = lstm_cell.zero_state(batch_size,dtype=tf.float32)
    
    
    outputs, states = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=_init_state,time_major=False)
    
    
    #hidden layer for output as the final results
    results =tf.matmul(states[1], weights['out'])+biases['out']
    
    #or
    #unpack to list[(batch,output)...] * steps
#     outputs = tf.unpack(tf.transpose(outputs, [1,0,2]))  #state is the last outputs
#     results = tf.matmul(outputs[-1], weights['out']+biases['out'])


    return results


pred = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1),tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    step = 0
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([train_op],feed_dict={
            x:batch_xs,
            y:batch_ys
        }
        )
        if step % 20 == 0:
            print(sess.run(accuracy,feed_dict={
                x:batch_xs,
                y:batch_ys
            }))
        step += 1

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.1171875
0.5078125
0.7734375
0.78125
0.8828125
0.8125
0.8203125
0.8828125
0.90625
0.953125
0.9140625
0.9453125
0.9453125
0.9765625
0.8984375
0.921875
0.9140625
0.9296875
0.921875
0.953125
0.953125
0.953125
0.921875
0.9375
0.9453125
0.9375
0.984375
0.96875
0.984375
0.984375
0.9375
0.9609375
0.9453125
0.9609375
0.9453125
0.953125
0.984375
0.953125
0.9296875
0.9453125
